# TFX interactive mode

Test of tfx in interactive mode

To download the dataset follow the [link](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers?hl=it)

In [1]:
from pathlib import Path
import tensorflow as tf
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, ExampleValidator, SchemaGen, Transform
from tfx.proto import example_gen_pb2
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft

data_path = Path("data/petfinder")

context = InteractiveContext()

2024-05-17 15:37:20.518149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=8),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
example_gen = CsvExampleGen(input_base=str(data_path),
              output_config=output)
statistic_gen = StatisticsGen(examples=example_gen.outputs['examples'])
schema_gen = SchemaGen(
    statistics=statistic_gen.outputs['statistics'],
    infer_feature_shape=True
)
example_validator = ExampleValidator(
    statistics=statistic_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file="/Users/enrico/PycharmProjects/tfx-learning/petfinder_tft_transform.py"
)


In [3]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [4]:
context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [5]:
context.show(statistic_gen.outputs['statistics'])

In [6]:
context.show(schema_gen.outputs['schema'])

In [7]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [9]:
context.show(example_validator.outputs['anomalies'])

In [10]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AdoptionSpeed',INT,required,,-
'Age',INT,required,,-
'Breed1',BYTES,required,,-
'Color1',STRING,required,,'Color1'
'Color2',STRING,required,,'Color2'
'Description',BYTES,required,,-
'Fee',INT,required,,-
'FurLength',STRING,required,,'FurLength'
'Gender',STRING,required,,'Gender'


,Values
Domain,
'Color1',"'Black', 'Brown', 'Cream', 'Golden', 'Gray', 'White', 'Yellow'"
'Color2',"'Brown', 'Cream', 'Golden', 'Gray', 'No Color', 'White', 'Yellow'"
'FurLength',"'Long', 'Medium', 'Short'"
'Gender',"'Female', 'Male'"
'Health',"'Healthy', 'Minor Injury', 'Serious Injury'"
'MaturitySize',"'Large', 'Medium', 'Small'"
'Sterilized',"'No', 'Not Sure', 'Yes'"
'Type',"'Cat', 'Dog'"
'Vaccinated',"'No', 'Not Sure', 'Yes'"


In [11]:
stats = tfdv.generate_statistics_from_csv(str(data_path / "petfinder-mini.csv"))
schema = tfdv.infer_schema(stats)

tfdv.display_schema(schema)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Type,Presence,Valency,Domain
Feature name,,,,
'Type',STRING,required,,'Type'
'Age',INT,required,,-
'Breed1',BYTES,required,,-
'Gender',STRING,required,,'Gender'
'Color1',STRING,required,,'Color1'
'Color2',STRING,required,,'Color2'
'MaturitySize',STRING,required,,'MaturitySize'
'FurLength',STRING,required,,'FurLength'
'Vaccinated',STRING,required,,'Vaccinated'


,Values
Domain,
'Type',"'Cat', 'Dog'"
'Gender',"'Female', 'Male'"
'Color1',"'Black', 'Brown', 'Cream', 'Golden', 'Gray', 'White', 'Yellow'"
'Color2',"'Brown', 'Cream', 'Golden', 'Gray', 'No Color', 'White', 'Yellow'"
'MaturitySize',"'Large', 'Medium', 'Small'"
'FurLength',"'Long', 'Medium', 'Short'"
'Vaccinated',"'No', 'Not Sure', 'Yes'"
'Sterilized',"'No', 'Not Sure', 'Yes'"
'Health',"'Healthy', 'Minor Injury', 'Serious Injury'"


In [12]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file="petfinder_tft_transform.py"
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying petfinder_tft_transform.py -> build/lib
installing to /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp0u6z7egm
running install
running install_lib
copying build/lib/petfinder_tft_transform.py -> /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp0u6z7egm
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'


/usr/local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Copying tfx_user_code_Transform.egg-info to /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp0u6z7egm/tfx_user_code_Transform-0.0+102d96282ca63076e3ab21328c5ed30d8f6f9d05779d9ea32c45cdad480132b1-py3.10.egg-info
running install_scripts
creating /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp0u6z7egm/tfx_user_code_Transform-0.0+102d96282ca63076e3ab21328c5ed30d8f6f9d05779d9ea32c45cdad480132b1.dist-info/WHEEL
creating '/var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmpctu_l6da/tfx_user_code_Transform-0.0+102d96282ca63076e3ab21328c5ed30d8f6f9d05779d9ea32c45cdad480132b1-py3-none-any.whl' and adding '/var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tmp0u6z7egm' to it
adding 'petfinder_tft_transform.py'
adding 'tfx_user_code_Transform-0.0+102d96282ca63076e3ab21328c5ed30d8f6f9d05779d9ea32c45cdad480132b1.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+102d96282ca63076e3ab21328c5ed30d8f6f9d05779d9ea32c45cdad480132b1.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+102d96282c

INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tfx-interactive-2024-05-17T15_37_32.800358-5xum5rrt/Transform/transform_graph/5/.temp_path/tftransform_tmp/11d486ca68274ece8509300e8c03c8f4/assets


INFO:tensorflow:Assets written to: /var/folders/42/59vqmp2x4xj9lb7str_hh_x80000gq/T/tfx-interactive-2024-05-17T15_37_32.800358-5xum5rrt/Transform/transform_graph/5/.temp_path/tftransform_tmp/11d486ca68274ece8509300e8c03c8f4/assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [13]:
context.show(transform.outputs['transformed_examples'])